# 第6回 ツリーモデル1

資産価格が従う確率微分方程式を2項ツリーモデルにより近似することを考える。モンテカルロ法と異なり、ツリーモデルを使うとアメリカンオプションの価格づけが容易にできる。また、モンテカルロ法よりも収束が早い。

In [2]:
# ライブラリのインポート
import numpy as np

## 2項ツリーの構築

確率微分方程式

$$ \tag{1}
 dS(t) = rS(t)dt + \sigma S(t)dW(t),\ S(0) = S_0
$$

または

$$
S(t) = S_0 \exp\left\{\left(r - \frac{\sigma^2}{2}\right)t + \sigma W(t) \right\}
$$

を、以下のような2項ツリーで近似することを考える。

時間区間$[0, T]$を$N$等分し、$\Delta t = T/N$、$t_i = i\Delta t$（$i=0,1,\dots,N$）とおく。

https://drive.google.com/file/d/1A5sNKRiwrFeaU43U7HruCsDQontL_uMK/view?usp=sharing


上昇確率$p$と上昇、下落した場合の収益$u$, $d$を(1)と整合するように決める必要がある。実は、つぎのように決めると(1)と整合的になる。

1. $u = e^{\sigma \sqrt{\Delta t}}$
1. $d = e^{-\sigma \sqrt{\Delta t}}$
1. $p=(e^{r\Delta t} - d)/(u - d)$

もしくは、つぎでもよい。

1. $u = \exp\left\{(r - \frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t}\right\}$
1. $d = \exp\left\{(r - \frac{\sigma^2}{2})\Delta t - \sigma \sqrt{\Delta t}\right\}$
1. $p = 1/2$


$t_i = i\Delta t$時点において $j$ 回（$j \leq i$） 株価が上昇したノード（これを $(i,j)$ノードと呼ぶ）における株価を$S_{i,j}$とおくと、
$S_{i,j}$は

$$
S_{i, j} = S_0u^{j}d^{i-j}
$$

によって決まる。（$i=0, 1, \dots, N$, $j=0, 1, \dots, i$）

そうすると満期$N\Delta t = T$における株価は $S_{N,j}$（$j=0,1,\dots,N$）となる。



## ヨーロピアンオプションの価格付け

権利行使価格が$K$のヨーロピアンコールオプションの価格づけを考える。コールオプションの支払いは

$$
(S(T) - K)^+
$$

だから、リスク中立化法によれば

$$
e^{-rT}E\left[(S(T) - K)^+\right]
$$

を計算すればよい。

2項ツリーの各ノードにおけるオプション価値を $C_{i,j}$ とおく。満期におけるオプション価値はオプションの支払いに等しくなるから

$$\tag{2}
C_{N,j} = (S_{N,j} - K)^+
$$

となる。2項ツリーモデルにおいては$(N,j)$ノードに到達する確率が

$$
 \begin{pmatrix}
 N\\
 j
\end{pmatrix}
p^j(1-p)^{N-j}
$$

となるため、オプション価格は

$$
 \sum_{j=0}^{N}
 \begin{pmatrix}
 N\\
 j
\end{pmatrix}
p^j(1-p)^{N-j}e^{-rT}(S_{N,j} - K)^+
$$

によって計算できる。ここで $\begin{pmatrix}N \\ j \end{pmatrix}$は$N$個の異なるものから$j$個を選ぶ方法の数を表す（二項係数という）。

またつぎのように求めることもできる。
(2)から始めて、$i\leq N-1$時点におけるオプションの価格を

$$
	C_{i,j} = e^{-r\Delta t}(pC_{i+1,j+1}+(1-p)C_{i+1,j}),\quad
	 i = N-1,N-2,\dots,0,\ j=0,1,\dots,i
$$

と後ろ向きにつぎつぎと計算していく。
そうすると$C_{0,0}$が求めたいオプション価格になる。

以上をまとめるとつぎのようになる。

1. 適切に決定した$u, d, p$を使って、株価のツリーを構築する。
$(i,j)$ノードにおける株価$S_{i,j}$は

$$
S_{i,j} = S_0u^jd^{i-j}
$$

である。
2. 満期時点におけるオプションの価値を計算する。コールオプションの場合

$$
C_{N,j} = (S_{N,j} - K)^+,\quad j=0,1,\dots,N
$$

とする。
3. オプションの価格を後ろ向きに計算する。つまり

$$
	C_{i,j} = e^{-r\Delta t}(pC_{i+1,j+1}+(1-p)C_{i+1,j})
$$

として、$C_{0,0}$が求めたいオプション価格になる。

以下がPythonのコードになる。

In [3]:
S0 = 100 # 株価、オプションのパラメータ
K = 99
T = 1
r = 0.01
sigma = 0.2

N = 500  # [0, T]の分割数
S = np.zeros((N+1, N+1)) # 株価を格納する変数
C = np.zeros((N+1, N+1))  # オプション価値を格納する変数

delta = T/N  # 時間幅
u = np.exp(sigma * np.sqrt(delta))  # 2項ツリーのパラメータ
d = np.exp(-sigma * np.sqrt(delta))
p = (np.exp(r * delta) - d) / (u - d)

# 株価ツリーの構築
for i in range(N+1):   
  for j in range(i+1):
    S[i, j] = S0 * u**j * d**(i - j)

# 満期におけるオプション価格
for j in range(N+1):
  C[N, j] = np.maximum(S[N, j] - K, 0)

# 後ろ向きにオプション価値を計算
for i in reversed(range(N)):
  for j in range(i+1):
    C[i, j] = np.exp(-r * delta) * (p * C[i+1, j+1] + (1 - p) * C[i+1, j])

print(f'{C[0, 0]:.3f}')

8.922


**演習問題**
以下のパラメータ

1. $u = \exp\left\{(r - \frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t}\right\}$
1. $d = \exp\left\{(r - \frac{\sigma^2}{2})\Delta t - \sigma \sqrt{\Delta t}\right\}$
1. $p = 1/2$

により2項ツリーを構築し、ヨーロピアンコールオプションの価格を求めよ。

In [4]:
S0 = 100 # 株価、オプションのパラメータ
K = 99
T = 1
r = 0.01
sigma = 0.2

N = 500  # [0, T]の分割数
S = np.zeros((N+1, N+1)) # 株価を格納する変数
C = np.zeros((N+1, N+1))  # オプション価値を格納する変数

delta = T/N  # 時間幅
u = np.exp((r - 0.5*sigma**2) * delta + sigma * np.sqrt(delta))  # 2項ツリーのパラメータ
d = np.exp((r - 0.5*sigma**2) * delta - sigma * np.sqrt(delta))
p = 0.5

# 株価ツリーの構築
for i in range(N+1):   
  for j in range(i+1):
    S[i, j] = S0 * u**j * d**(i - j)

# 満期におけるオプション価格
for j in range(N+1):
  C[N, j] = np.maximum(S[N, j] - K, 0)

# 後ろ向きにオプション価値を計算
for i in reversed(range(N)):
  for j in range(i+1):
    C[i, j] = np.exp(-r * delta) * (p * C[i+1, j+1] + (1 - p) * C[i+1, j])

print(f'{C[0, 0]:.3f}')

8.915


## アメリカンオプションの価格付け

満期においてのみ権利行使ができるオプションをヨーロピアンオプションと呼ぶのに対し、
満期までの間に**いつでも**権利行使ができるオプションを
**アメリカンオプション**と呼ぶ。

オプションの支払い関数をとすると$f(x)$（行使価格が$K$のコールオプションの場合$h(x) = (x - K)^+$、プットオプションの場合は$h(x) = (K - x)^+$）

アメリカンオプションの価格$V_0$は

$$
V_0 = \sup_{\tau}E[e^{-r\tau}f(S(\tau))]       
$$

によって与えられる。
ここで $\tau$は停止時刻と呼ばれるものである。

このとき２項ツリーモデルにおいて$(i,j)$ノードおけるアメリカンオプションの価値$V_{i,j}$はつぎのような**動的計画方程式**を満たすことが
知られている。

$$
V_{i,j} = \max\left\{f(S_{i,j}), e^{-r\Delta t}(pV_{i+1,j+1}+(1-p)V_{i+1,j})\right\}
$$

ここで$f(S_{i,j})$はその時点でオプションを行使した際の価値、
$e^{-r\Delta t}(pV_{i+1,j+1}+(1-p)V_{i+1,j})$はオプションを行使しなかった場合に得られる価値を表す。

株式に配当がない場合はアメリカンコールオプションは満期時点において権利行使をするのが最適、すなわちアメリカンコールオプションとヨーロピアンコールオプションの価値は同じであることが知られている。

そこで、支払いが$(K - S(T))^+$であるアメリカンプットオプションの価格を２項ツリーを使って近似する。




In [8]:
S0 = 100 # 株価、オプションのパラメータ
K = 99
T = 1
r = 0.01
sigma = 0.2

N = 100  # [0, T]の分割数
S = np.zeros((N+1, N+1)) # 株価を格納する変数
V = np.zeros((N+1, N+1)) # オプション価値を格納する変数

delta = T/N # 時間幅
u = np.exp(sigma * np.sqrt(delta))  # 2項ツリーのパラメータ
d = np.exp(-sigma * np.sqrt(delta))
p = (np.exp(r * delta) - d) / (u - d)

# 株価ツリーの構築
for i in range(N+1):
  for j in range(i+1):
    S[i, j] = S0 * u**j * d**(i - j)

# 満期におけるオプション価格
for j in range(N+1):
  V[N, j] = np.maximum(K - S[N, j], 0)

# 後ろ向きにオプション価値を計算
for i in reversed(range(N)):
  for j in range(i+1):
    V[i, j] = np.maximum(K - S[i, j], np.exp(-r * delta) * (p * V[i+1, j+1] + (1 - p) * V[i+1, j]))

print(f'{V[0, 0]:.3f}')

7.013


**演習問題**
アメリカンコールオプションの価格を2項ツリーを使って計算し、ヨーロピアンコールオプションの価格を同じになることを確認してみよ。

In [9]:
S0 = 100 # 株価、オプションのパラメータ
K = 99
T = 1
r = 0.01
sigma = 0.2

N = 100  # [0, T]の分割数
S = np.zeros((N+1, N+1)) # 株価を格納する変数
V = np.zeros((N+1, N+1)) # オプション価値を格納する変数

delta = T/N # 時間幅
u = np.exp(sigma * np.sqrt(delta))  # 2項ツリーのパラメータ
d = np.exp(-sigma * np.sqrt(delta))
p = (np.exp(r * delta) - d) / (u - d)

# 株価ツリーの構築
for i in range(N+1):
  for j in range(i+1):
    S[i, j] = S0 * u**j * d**(i - j)

# 満期におけるオプション価格
for j in range(N+1):
  V[N, j] = np.maximum(S[N, j] - K, 0)

# 後ろ向きにオプション価値を計算
for i in reversed(range(N)):
  for j in range(i+1):
    V[i, j] = np.maximum(S[i, j] - K, np.exp(-r * delta) * (p * V[i+1, j+1] + (1 - p) * V[i+1, j]))

print(f'{V[0, 0]:.3f}')

8.928
